#  Adding RBF layer to prevent adversarial examples

In [10]:
from keras.datasets import mnist
from keras.utils import np_utils

In [11]:
# prepare MNIST data set 

(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("Loaded MNIST test data.")

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)


Loaded MNIST test data.


### Evaluate MLP network

In [12]:
from eval_adversarial import eval

In [13]:
eval("mlp", X_train, Y_train, X_test, Y_test)

Load model ... ok


/home/petra/cleverhans/cleverhans/utils_th.py:171: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "


Test accuracy on legitimate test examples: 0.9855


/home/petra/cleverhans/cleverhans/attacks_th.py:36: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "
/home/petra/cleverhans/cleverhans/utils_th.py:76: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "
/home/petra/cleverhans/cleverhans/utils_th.py:231: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "


Test accuracy on adversarial examples: 0.026


### Create new model by adding RBF layer 

In [6]:
from mnist_add_rbf import add_rbf_layer
from keras.models import model_from_json

# load original MLP
with open("models/mlp.json") as f:
    mlp = model_from_json(f.read()) 
mlp.load_weights("models/mlp_weights.h5") 

In [16]:
# create new model by adding RBF layer  
mlp_rbf =  add_rbf_layer(mlp, 2.0, X_train, Y_train, X_test, Y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
__________

In [17]:
# save new model 
json_string = mlp_rbf.to_json()
with open("models/mlprbf.json","w") as f:
    f.write(json_string) 
mlp_rbf.save_weights("models/mlprbf_weights.h5")


### Eval new model  

In [18]:
# eval new model 
eval("mlprbf", X_train, Y_train, X_test, Y_test)

Load model ... ok


/home/petra/cleverhans/cleverhans/utils_th.py:171: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "


Test accuracy on legitimate test examples: 0.9808


/home/petra/cleverhans/cleverhans/attacks_th.py:36: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "
/home/petra/cleverhans/cleverhans/utils_th.py:76: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "
/home/petra/cleverhans/cleverhans/utils_th.py:231: UserWarning: CleverHans support for Theano is deprecated and will be dropped on 2017-11-08.
  warnings.warn("CleverHans support for Theano is deprecated and "


Test accuracy on adversarial examples: 0.745
